In [1]:
%matplotlib inline
import matplotlib
import pandas
import matplotlib.pyplot as plt

In [2]:
salary_train = pandas.read_csv('salary-train.csv')
salary_test= pandas.read_csv('salary-test-mini.csv')

In [3]:
salary_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [4]:
salary_train.describe()

,SalaryNormalized
count,60000.000000
mean,34084.931500
std,17593.539646
min,5000.000000
25%,21500.000000
50%,30000.000000
75%,42500.000000
max,200000.000000


In [5]:
salary_test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [6]:
salary_train['FullDescription'] = salary_train['FullDescription'].str.lower()
salary_test['FullDescription'] = salary_test['FullDescription'].str.lower()

In [7]:
salary_test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,Milton Keynes,contract,NaN
1,a web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [8]:
salary_train['FullDescription'] = salary_train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
salary_test['FullDescription'] = salary_test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [9]:
from sklearn import feature_extraction

In [10]:
txt = feature_extraction.text.TfidfVectorizer(max_df=1.0, min_df=5)

In [11]:
print len(salary_train['LocationNormalized'])

60000


In [12]:
train = txt.fit_transform(salary_train['FullDescription'])

In [25]:
test = txt.transform(salary_test['FullDescription'])

In [26]:
salary_train['LocationNormalized'].fillna('nan', inplace=True)
salary_train['ContractTime'].fillna('nan', inplace=True)

In [27]:
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
X_train_categ = enc.fit_transform(salary_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(salary_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [28]:
import scipy
h_train = scipy.sparse.hstack([train, X_train_categ])
h_test = scipy.sparse.hstack([test, X_test_categ])

In [29]:
from sklearn import linear_model

In [30]:
clf = linear_model.Ridge(alpha=1)

In [31]:
import numpy as np

In [32]:
h_train

<60000x24627 sparse matrix of type '<type 'numpy.float64'>'
	with 8485759 stored elements in COOrdinate format>

In [33]:
h_test

<2x24627 sparse matrix of type '<type 'numpy.float64'>'
	with 304 stored elements in COOrdinate format>

In [34]:
clf.fit(h_train, salary_train['SalaryNormalized']) 

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [35]:
result = clf.predict(h_test)

In [36]:
print round(result[0],2), round(result[1],2)

56572.72 37201.22
